<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/Sihyun/data_exploration_judges3(normalized_winner_April_20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team

/content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team


Import libraries

In [5]:
import pandas as pd
import numpy as np
import altair as alt
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn import preprocessing
import seaborn as sns

In [6]:
!pip install --upgrade altair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [7]:
#!ls

In [8]:
# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)



/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
webscraping.keys()
# webscraping['Events and8']

dict_keys(['Events and8', 'Events', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

Events details

In [10]:
#Load battle winners
#threefold
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
#Trivium
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
#traditional
traditional = webscraping['Battles and8 Traditional']
traditional.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
traditional[['Event', 'Category']] = traditional[['Event', 'Category']].fillna(method='ffill')

In [11]:
tr=traditional[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
th=threefold[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
tv=trivium[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()

dataframes = [('traditional', tr), ('threefold', th), ('trivium', tv)]

# Create an empty list to store modified dataframes
appended_dfs = []

# Iterate to create a combined dataframe
for name, dframe in dataframes:
    dframe['Original_DF'] = name
    dframe['Battle_ID'] = dframe.index
    appended_dfs.append(dframe)
appendedBattles = pd.concat(appended_dfs, ignore_index=True)

In [12]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

Events = webscraping['Events'][1:].copy()
new_header = Events.iloc[0]
Events = Events.drop(1)
Events.columns = new_header
Events.rename(columns={'Name':'Event'}, inplace=True)
Events.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF, Events], ignore_index=True)


In [13]:
#add battle date
appendedBattles=pd.merge(appendedBattles, combined_events[['Date', 'Event']], on='Event', how='left')


In [14]:
appendedBattles.tail(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID,Date
2627,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,trivium,1524,2017-06-10 00:00:00
2628,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,D-Matt,Rerunn,D-Matt,trivium,1525,2017-06-10 00:00:00


Threefold Battles

In [15]:
#threefold battles
# Index threefold.at[238, 'battle rounds'] = 2 update value 2 from 4(wrong!)

threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
threefold.head(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Baruzzo,Elpablix,Baruzzo,3,3,2,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Yoga,Faby,Faby,2,3,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Change new names
threefold.rename(columns = {"breaker 1": "breaker 1 (red)", "breaker 2": "breaker 2 (blue)"}, inplace=True)


In [17]:
threefold.columns

Index(['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner',
       'battle rounds', 'number of judges', 'breaker 1 round wins',
       'breaker 2 round wins', 'breaker 1 vote count',
       ...
       'r3j7arti', 'r3j7inte', 'r3j8over', 'r3j8phys', 'r3j8arti', 'r3j8inte',
       'r3j9over', 'r3j9phys', 'r3j9arti', 'r3j9inte'],
      dtype='object', length=128)

In [18]:

# Using DataFrame.insert() to add a column
threefold.insert(2, "Method", ['three']*len(threefold), True)

In [19]:
# Create 'Date' Column with combined_events

def Date_Added(df, combined_df):
  lst = []
  for i in range(len(df)):
    lst.append(combined_df[combined_df['Event']==df['Event'].iloc[i]]['Date'].iloc[0])


  # Using DataFrame.insert() to add a column
  df.insert(0, "Date", lst, True)
  return df


In [20]:
threefold = Date_Added(threefold, appendedBattles)

Trivium Battles

In [21]:
test_trivium=webscraping['Battles Trivium']
test_trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
test_trivium[['Event', 'Category']]= test_trivium[['Event', 'Category']].fillna(method='ffill') # fill values by Event's name

In [22]:
# Using DataFrame.insert() to add a column
test_trivium.insert(2, "Method", ['trivium']*len(test_trivium), True)

In [23]:
test_trivium = Date_Added(test_trivium, combined_events)

In [24]:
test_trivium.head(2)

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Ami,671,671,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Sunny,Anti,Anti,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Merge with Trivium & Threefold
combined_tv_th = pd.concat([test_trivium, threefold], ignore_index=True)

In [26]:
combined_tv_th.tail()

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r3j6inte,r3j7phys,r3j7arti,r3j7inte,r3j8phys,r3j8arti,r3j8inte,r3j9phys,r3j9arti,r3j9inte
1894,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Wigor,El Nino,Wigor,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1895,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Mighty Jimm,Nasty,Mighty Jimm,3,7,2,...,0.33,-3.67,-4.33,-3.00,NaN,NaN,NaN,NaN,NaN,NaN
1896,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Daniel,Mass,Mass,3,7,1,...,0.67,2.33,3.00,3.67,NaN,NaN,NaN,NaN,NaN,NaN
1897,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Quake,Flo,Quake,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1898,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Neosan,Lilbom,Lilbom,2,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# judge_df

print(tabulate(combined_tv_th.tail(10), headers='keys', tablefmt='psql'))

+------+---------------------+--------------------------------------------------------+-------------------------------------+----------+-------------------+--------------------+-------------+-----------------+--------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------

# Threefold -
Components :

*  'Physical Quality',
*  'Artistic Quality',
*  'Interpretive Quality'

# Trivium -
Components :

*  'Technique',
*  'Variety',
*  'Performance',
*  'Musicality',
*  'Creativity',
*  'Personality'

Create Dataframe -'Event', 'Category', 'Round number','Name', 'Overall', 'Technique', 'Variety', 'Performance', 'Musicality', 'Creativity', 'Personality'

Dictionary


*   **Date** = date of event (datetime)
*   **Event** = name of event (string)
*   **Category** = name of round (string)
*   **Round Number** = the number of each round (r1 means the first round for battle)
*   **Judge** = name of judge (string)
*   **Overall** = Sum of judge scores (float)
*   **Technique(Physical)** = judge scores for techique(trivium) or physical quality(threefold) (float)
*   **Variety(Artistic)** = judge scores for variety(trivium) or artistic quality(threefold) (float)
*   **Performance(Interpretive)** = judge scores for performance(trivium) or interpretive quality(threefold) (float)
*   **musicalirty** = judge scores for musicality (float)
*   **Creativity** = judge scores for creativity (float)
*   **Personality** = judge scores for personality (float)


In [28]:
columns = ['Date', 'Event', 'Category', 'Round_Number','Judge', 'Winner_In_Round', 'Overall', 'Technique(Physical)', 'Variety(Artistic)', 'Performance(Interpretive)', 'Musicality', 'Creativity', 'Personality']
judge_df = pd.DataFrame(columns=columns)

In [29]:
judge_df

,Date,Event,Category,Round_Number,Judge,Winner_In_Round,Overall,Technique(Physical),Variety(Artistic),Performance(Interpretive),Musicality,Creativity,Personality


# *Group* by Judge

In [30]:
event_lst = list(combined_tv_th['Event'].unique())

total_judges_lst = [] # Save the entire judges

total_breakers_lst = [] # Save the entire breakers

In [31]:
# If there is any change, you must run dataframe judge_df first!
# Index number is initialized
index_num = 0

for event in event_lst:
  temp_lst = [] # empty list
  temp_df = combined_tv_th[combined_tv_th['Event']==event] # Take dataframe by each event

  date = temp_df['Date'].iloc[0]

  category_lst = temp_df['Category'].unique()
  #print(category_lst)

  for category in category_lst:

    cat_temp_df = temp_df[temp_df['Category']==category] # Take dataframe by category in each event
    # print(cat_temp_df['battle rounds'].unique())
    # print('')


    for k in range(len(cat_temp_df)):
      num_judges = cat_temp_df['number of judges'].iloc[k]
      judges_lst = list(cat_temp_df.filter(like="judge ").iloc[k, :num_judges]) # Judges list
      num_round = cat_temp_df['battle rounds'].iloc[k] # number of round

      total_judges_lst = list(set(total_judges_lst + judges_lst))

      num_breakers = 2 # number of breakers
      breakers_lst = list(cat_temp_df.filter(like="breaker ").iloc[k, :num_breakers]) # breakers list

      # convert int to str!!
      if 671 in breakers_lst:
        breakers_lst[0] = str(breakers_lst[0])
        breakers_lst[1] = str(breakers_lst[1])

      total_breakers_lst = list(set(total_breakers_lst + breakers_lst))

      for i in range(num_round):
        round = "r"+str(i+1) # Round number

        for j in range(num_judges):
          temp_lst.append(date)
          temp_lst.append(event)
          temp_lst.append(category)
          temp_lst.append(round)

          temp_lst.append(judges_lst[j])

          each_r_j_num = round+'j'+str(j+1)
          over = each_r_j_num+'over'

          # if trivium data
          if cat_temp_df['Method'].iloc[k]=='trivium':
            # Score list - Overall, Tech, Variety, Performance, Musicality, Creative, Personality
            tech = each_r_j_num +'tech'
            vari = each_r_j_num +'vari'
            perf = each_r_j_num +'perf'
            musi = each_r_j_num +'musi'
            crea = each_r_j_num +'crea'
            pers = each_r_j_num +'pers'

            a_l = [over, tech, vari, perf, musi, crea, pers]
            s_lst = list(cat_temp_df[a_l].iloc[k])

          else:
            s_lst = [] # Save scores(by judges)

            phys = each_r_j_num +'phys'
            arti = each_r_j_num +'arti'
            inte = each_r_j_num +'inte'

            a_l = [over, phys, arti, inte]

            s_lst = list(cat_temp_df[a_l].iloc[k]) + [np.nan]*3

          # Append winner breaker by Overall score
          # if Overall > 0, blue(breaker2)
          temp_lst.append(np.where(s_lst[0]>0, str(breakers_lst[1]), str(breakers_lst[0])))

          judge_df.loc[index_num] = temp_lst+ s_lst
          index_num = index_num + 1

          temp_lst = [] # reset the list


#temp_df



In [32]:

print(tabulate(judge_df.tail(100), headers='keys', tablefmt='psql'))

+-------+---------------------+--------------------------------------------------------+-------------------------------------+----------------+-------------+-------------------+-----------+-----------------------+---------------------+-----------------------------+--------------+--------------+---------------+
|       | Date                | Event                                                  | Category                            | Round_Number   | Judge       | Winner_In_Round   |   Overall |   Technique(Physical) |   Variety(Artistic) |   Performance(Interpretive) |   Musicality |   Creativity |   Personality |
|-------+---------------------+--------------------------------------------------------+-------------------------------------+----------------+-------------+-------------------+-----------+-----------------------+---------------------+-----------------------------+--------------+--------------+---------------|
| 20776 | 2023-04-06 00:00:00 | WDSF BfG International Series - 

In [33]:
total_breakers_lst.sort()
total_judges_lst.sort()

In [34]:
len(total_breakers_lst)

738

In [35]:
len(total_judges_lst)

80

## Winner compared with raw data and normalized data

In [36]:
# new dataframe for each round

columns = ['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner',
       'round', 'number of judges', 'j1over', 'j2over', 'j3over', 'j4over', 'j5over', 'j6over', 'j7over', 'j8over', 'j9over', 'total']
new_df = pd.DataFrame(columns=columns)

Dictionary



*   **Event** = name of event (string)
*   **Category** = name of round (string)
*   **breaker 1 (red)** = breaking dancer 1 (string)
*   **breaker 2 (blue)** = breaking dancer 2 (string)
*   **winner** = winner of round (string)
*   **round** = the number of each round (r1 means the first round for battle)
*   **number of judges** = number of judges (string)
*   **j1over** = overall score of judge 1 (float)
*   **j2over** = overall score of judge 2 (float)
*   **j3over** = overall score of judge 3 (float)
*   **j4over** = overall score of judge 4 (float)
*   **j5over** = overall score of judge 5 (float)
*   **j6over** = overall score of judge 6 (float)
*   **j7over** = overall score of judge 7 (float)
*   **j8over** = overall score of judge 8 (float)
*   **j9over** = overall score of judge 9 (float)
*   **total** = sum of overall scores (float)

In [37]:
j=0
t_lst = np.array(test_trivium.filter(like="over").iloc[0, 0+9*j:9+9*j])
t_lst

array([ 14.93, -10.67,  -0.4 ,   6.4 ,  -4.47,  -3.47, -12.67,   1.73,
        -6.8 ])

In [38]:
# code to replace all negative value with 0
t_lst[t_lst>0] = np.nan
print(t_lst)

t_lst[np.isnan(t_lst)] = 0



a = t_lst.sum()

lst3 = []

lst3.append(a)

lst3

[   nan -10.67  -0.4     nan  -4.47  -3.47 -12.67    nan  -6.8 ]


[-38.48]

In [54]:
#event_temp_df['Overall'] = event_temp_df['Overall'].abs()

# new dataframe for each round

columns = ['Event', 'Category', 'winner', 'normalized winner',
       'round', 'number of judges', 'breaker', 'j1over', 'j2over', 'j3over',
           'j4over', 'j5over', 'j6over', 'j7over', 'j8over', 'j9over', 'total']
new_df = pd.DataFrame(columns=columns)

In [55]:
cols_diff = ['Event', 'Category', 'round', 'winner', 'normalized winner',
      'difference']
df3_difference = pd.DataFrame(columns=cols_diff)

summary_df3_difference = pd.DataFrame(columns=cols_diff)


In [60]:
temp_df = combined_tv_th.copy()

In [57]:
#temp_df = temp_df.head(128)

In [61]:
# Keep it for the reference!

index_num = 0 # index number for new_df

min_max_scaler = preprocessing.MinMaxScaler() # to normalize overall scores

for i in range(len(temp_df)):
  # print(type(test_trivium.iloc[i]))
  temp_lst = []
  lst3 = []

  num_round = temp_df.iloc[i]['battle rounds'] # number of round

  for j in range(num_round):
    for k in range(2):
      temp_lst.append(temp_df.iloc[i]['Event'])
      temp_lst.append(temp_df.iloc[i]['Category'])
      temp_lst.append(temp_df.iloc[i]['winner'])
      temp_lst.append(np.nan) # set np.nan at 'normalized winner'

      round = "r"+str(j+1) # Round number
      temp_lst.append(round)

      temp_lst.append(temp_df.iloc[i]['number of judges'])

      # Score list - Overall scores
      s_lst = np.array(temp_df.filter(like="over").iloc[i, 0+9*j:9+9*j])
      # filter nan to zero
      #s_lst = np.nan_to_num(s_lst)

      if k==0: # first loop
        # code to replace all positive value with np.nan
        s_lst[s_lst>0] = np.nan
        temp_lst.append(temp_df.iloc[i]['breaker 1 (red)'])
      else : # k==1
        # code to replace all negative value with np.nan
        s_lst[s_lst<0] = np.nan
        temp_lst.append(temp_df.iloc[i]['breaker 2 (blue)'])

      # print(temp_lst)
      # print('')

      s2_lst = s_lst
      s2_lst[np.isnan(s2_lst)] = 0

      lst3.append(s2_lst.sum()) # Save total

      new_df.loc[index_num] = temp_lst+ list(s_lst) + lst3
      index_num = index_num + 1

      temp_lst = [] # reset the list
      lst3 = []

  if i+1 == len(temp_df): # out-of-bounds
    continue

  # Separated by each category becase we need to keep the same judges in the category
  elif temp_df.iloc[i]['Category'] != temp_df.iloc[i+1]['Category']:

    # Original dataframe
    new_df.iloc[:,7:] = new_df.iloc[:,7:].abs()

    #
    # new dataframe for each round
    #
    print(tabulate(new_df, headers='keys', tablefmt='psql'))

    df2_normalized=new_df.copy()

    # Only columns to normalized - from 'r1over' to 'r9over'
    x = df2_normalized.iloc[:,7:].values
    x_scaled = min_max_scaler.fit_transform(x)
    df_temp = pd.DataFrame(x_scaled)
    df2_normalized.iloc[:,7:] = df_temp

    df2_normalized['total'] = df2_normalized.fillna(0)['j1over'] + df2_normalized.fillna(0)['j2over'] + df2_normalized.fillna(0)['j3over'] + df2_normalized.fillna(0)['j4over'] + df2_normalized.fillna(0)['j5over'] + df2_normalized.fillna(0)['j6over'] + df2_normalized.fillna(0)['j7over'] + df2_normalized.fillna(0)['j8over'] + df2_normalized.fillna(0)['j9over']


    index_num2 = 0

    for n in range(0,len(df2_normalized), 2): # every even number
      #print(df2_normalized['total'].iloc[n])
      temp_lst_diff = []
      temp_lst_diff.append(df2_normalized['Event'].iloc[n])
      temp_lst_diff.append(df2_normalized['Category'].iloc[n])
      temp_lst_diff.append(df2_normalized['round'].iloc[n])
      winner = df2_normalized['winner'].iloc[n]
      temp_lst_diff.append(winner)

      if df2_normalized['total'].iloc[n] > df2_normalized['total'].iloc[n+1]:
        # Select 'red'
        normalized_winner = df2_normalized['breaker'].iloc[n]
      else :
        # Select 'blue'
        normalized_winner = df2_normalized['breaker'].iloc[n+1]

      temp_lst_diff.append(normalized_winner)

      if winner != normalized_winner:
        temp_lst_diff.append('Yes')
      else :
        temp_lst_diff.append('No')

      df3_difference.loc[index_num2] = temp_lst_diff
      index_num2 = index_num2 + 1

    # Find out normalized table
    print(tabulate(df2_normalized, headers='keys', tablefmt='psql'))

    # concatenating df1 and df2 along rows
    summary_df3_difference = pd.concat([summary_df3_difference, df3_difference], axis=0)
    print(tabulate(df3_difference, headers='keys', tablefmt='psql'))

    # new dataframe for each round
    new_df = pd.DataFrame(columns=columns)

    df3_difference = pd.DataFrame(columns=cols_diff)

  else:
    continue





Streaming output truncated to the last 5000 lines.
| 5626 | YOG Continental Qualifier 2017 USA | B-Girls Top 4 Round Robin | Connie   |                 nan | r2      |                  5 | Connie    |     0    |     0.8  |     0    |     4    |     2    |        0 |        0 |        0 |        0 |    6.8  |
| 5627 | YOG Continental Qualifier 2017 USA | B-Girls Top 4 Round Robin | Connie   |                 nan | r2      |                  5 | Emma      |     3.6  |     0    |     0.53 |     0    |     0    |        0 |        0 |        0 |        0 |    4.13 |
| 5628 | YOG Continental Qualifier 2017 USA | B-Girls Top 4 Round Robin | Esya     |                 nan | r1      |                  5 | Esya      |     0.47 |     1.33 |     2.07 |     1    |     4    |        0 |        0 |        0 |        0 |    8.87 |
| 5629 | YOG Continental Qualifier 2017 USA | B-Girls Top 4 Round Robin | Esya     |                 nan | r1      |                  5 | Nadjaya   |     0    |     0    | 

In [68]:
summary_df3_difference.groupby(['round', 'difference']).agg(difference_count=('difference','count'))

difference_count
round difference                  
r1    No                      1852
      Yes                      287
r2    No                      1472
      Yes                      251
r3    No                       230
      Yes                       33
r4    No                        74
      Yes                       24
r5    No                         3
      Yes                        1

In [70]:
sum_df = summary_df3_difference.groupby(['round', 'difference']).agg(difference_count=('difference','count'))

In [74]:
sum_df = sum_df.reset_index()

In [75]:
sum_df

,round,difference,difference_count
0,r1,No,1852
1,r1,Yes,287
2,r2,No,1472
3,r2,Yes,251
4,r3,No,230
5,r3,Yes,33
6,r4,No,74
7,r4,Yes,24
8,r5,No,3
9,r5,Yes,1


In [80]:
alt.Chart(sum_df).mark_bar().encode(
    x='round',
    y='difference_count',
    color='difference'
)

alt.Chart(...)

In [95]:
# Normalize difference in each round

alt.Chart(summary_df3_difference).transform_aggregate(
    count='count()',
    groupby=['round', 'difference']
).transform_joinaggregate(
    total='sum(count)',
    groupby=['round']
).transform_calculate(
    frac=alt.datum.count / alt.datum.total
).mark_bar().encode(
    x="round:O",
    y=alt.Y('count:Q', stack="normalize", axis=alt.Axis(title="Percent", format="%")),
    color='difference:N',
    tooltip=[
        alt.Tooltip('count:Q', title="Total Difference"),
        alt.Tooltip('frac:Q', title="Percentage of Difference", format='.0%')
    ]
)

alt.Chart(...)

# Thanks you!